In [44]:
import tensorflow as tf
import pandas as pd

from pprint import pprint as pp
from IPython.display import display

### Data

데이터는 [UCI Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/Census+Income)에서 다운받을수 있습니다.<br>
1년 소득이 50k/yr (약 5500만원)넘는 사람들을 인구조사 데이터에 기반해서 예측을 합니다.

In [91]:
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "gender",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "income_bracket"]
train_data = pd.read_csv('../../data/census-income/adult.data.txt', names=COLUMNS, skipinitialspace=True)
test_data = pd.read_csv('../../data/census-income/adult.test.txt', names=COLUMNS, skipinitialspace=True)

test_data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


### Data Preprocessing with Categorical Data and Continuous Data

**Feature Column**이란 original dataframe에 있는 raw columns(**base feature columns**라고 부릅니다) 또는 transformations을 통해서 새로 만들어진 **derived feature columns** 라고 합니다. feature columns을 제대로 선택하고 다루는것이 첫번째 machine learning을 잘하기 위한 방법입니다.



* **label**: 1은 연소득 5500만원 넘는 인간들..

In [94]:
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation",
                       "relationship", "race", "gender", "native_country"]
CONTINUOUS_COLUMNS = ["age", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
LABEL_COLUMN = 'label'

# 소득에 따라 label을 만듭니다. 0은 5500만원 미만.. 1은 5500만원 이상
train_data[LABEL_COLUMN] = train_data['income_bracket'].apply(lambda x: '>50K' in x).astype('int')
print(test_data[test_data['income_bracket'].isnull()]['income_bracket'])
test_data[LABEL_COLUMN] = test_data['income_bracket'].apply(lambda x: '>50K' in x).astype('int')

def input_fn(df):
    continuous_cols = {k: tf.constant(df[k].values, name=k) for k in CONTINUOUS_COLUMNS}
    categorical_cols = {k: tf.SparseTensor(
        indices=[[i, 0] for i in range(df[k].size)], # train의 경우 32561개의 [[0, 0], [1, 0], [2, 0], ...] 만들어짐
        values=df[k].values,
        dense_shape=[df[k].size, 1], # [32561, 1]
    ) for k in CATEGORICAL_COLUMNS}
    
    feature_cols = dict(**continuous_cols, **categorical_cols)
    label = tf.constant(df[LABEL_COLUMN].values)
    return feature_cols, label
    
input_fn(train_data)

Series([], Name: income_bracket, dtype: object)


({'age': <tf.Tensor 'age_11:0' shape=(32561,) dtype=int64>,
  'capital_gain': <tf.Tensor 'capital_gain_11:0' shape=(32561,) dtype=int64>,
  'capital_loss': <tf.Tensor 'capital_loss_11:0' shape=(32561,) dtype=int64>,
  'education': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f7a9fcc3c50>,
  'education_num': <tf.Tensor 'education_num_11:0' shape=(32561,) dtype=int64>,
  'gender': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f7a9b5159e8>,
  'hours_per_week': <tf.Tensor 'hours_per_week_11:0' shape=(32561,) dtype=int64>,
  'marital_status': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f7a9ee0b358>,
  'native_country': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f7a9a4492b0>,
  'occupation': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f7aa074b940>,
  'race': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f7a9f584f60>,
  'relationship': <tensorflow.python.framework.sparse_tensor.SparseTensor

_SparseColumnKeys(column_name='gender', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('female', 'male'), num_oov_buckets=0, vocab_size=2, default_value=-1), combiner='sum', dtype=tf.string)